In [79]:
from typing import List, Dict, Any
import json
import requests
import aiohttp
import sseclient
import time


FIREWORKS_API_ENDPOINT = "https://api.fireworks.ai/inference/v1/chat/completions"
TOGETHER_API_ENDPOINT = "https://api.together.xyz/v1/chat/completions"
OPENAI_API_ENDPOINT = "https://api.openai.com/v1/chat/completions"
CLAUDE_API_ENDPOINT = "https://api.anthropic.com/v1/messages"
# HEADERS = {
#   "accept": "application/json",
#   "content-type": "application/json",
# }
HEADERS = {
  "accept": "application/json",
  "content-type": "application/json",
  "anthropic-version": "2023-06-01",
}


class LLMEngine:
  def __init__(self, api_endpoint: str, api_key: str):
    self.api_endpoint = api_endpoint
    self.headers = HEADERS.copy()
    if self.api_endpoint == CLAUDE_API_ENDPOINT:
        self.headers.update({"x-api-key": api_key})
    else:
        self.headers.update({"Authorization": f"Bearer {api_key}"})

  def generate(
      self,
      messages: List[Dict[str, str]],
      model: str,
      temperature: float = 0.0,
      top_p: float = 1.0,
      max_new_tokens: int = 2048,
      stop_sequences: List[str] = None,
      **kwargs
  ):
    if self.api_endpoint == CLAUDE_API_ENDPOINT:
        payload = {
          "model": model,
          "messages": messages,
          "max_tokens": max_new_tokens,
          "temperature": temperature,
          "top_p": top_p,
          "top_k": 1,
          "stream": False,
        }
        response = requests.post(self.api_endpoint, json=payload, headers=self.headers)
        response.raise_for_status()
        return response.json()['content'][0]['text']
    else:
        payload = {
          "model": model,
          "messages": messages,
          "max_tokens": max_new_tokens,
          "temperature": temperature,
          "top_p": top_p,
          "stream": False,
          "stop": stop_sequences,
        }
        response = requests.post(self.api_endpoint, json=payload, headers=self.headers)
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']

  async def agenerate(
    self,
    messages: List[Dict[str, str]],
    model: str,
    temperature: float = 0.0,
    top_p: float = 1.0,
    max_new_tokens: int = 2048,
    stop_sequences: List[str] = None,
    **kwargs
  ):
    if self.api_endpoint == CLAUDE_API_ENDPOINT:
        payload = {
          "model": model,
          "messages": messages,
          "max_tokens": max_new_tokens,
          "temperature": temperature,
          "top_p": top_p,
          "top_k": 1,
          "stream": False,
        }
        async with aiohttp.ClientSession() as session:
          async with session.post(self.api_endpoint, json=payload, headers=self.headers) as response:
            response.raise_for_status()
            response_json = await response.json()
            return response_json['content'][0]['text']
    else:
        payload = {
          "model": model,
          "messages": messages,
          "max_tokens": max_new_tokens,
          "temperature": temperature,
          "top_p": top_p,
          "stream": False,
          "stop": stop_sequences,
        }
        async with aiohttp.ClientSession() as session:
          async with session.post(self.api_endpoint, json=payload, headers=self.headers) as response:
            response.raise_for_status()
            response_json = await response.json()
            return response_json['choices'][0]['message']['content']


In [80]:
api_key = "<insert api key>"
engine = LLMEngine(OPENAI_API_ENDPOINT, api_key)


In [81]:
DETECTION_PROMPT = """
Detect if the given Vietnamese text contains any English words. If English words are present, classify each as either a loanword, code-mixed indexicality, or code-switching, then translate the entire text into English.

# Steps
1. *Detection*: Detect whether the given text contains English words.
2. *Classification*: If English words are detected, classify each word as either a "loanword" (a word borrowed from English, adapted into Vietnamese); "code-mixed-indexicality" (word- or phrase-level stylistic language choice that signal belongingness to a social subgroup e.g. Gen Z, academics, businessperson, etc); or "code-switching" (word or phrase where Vietnamese-English language switch are seamless and does not represent personal/speaker linguistic style).  
3. *Translation*: Translate the entire text into English accurately.

# Output Format
- *Detection Result*: Output "True" if English words are present, "0" otherwise.
- *Classification*: If applicable, provide a list of English words with their classification as either "loanword" or "code-mixed-indexicality" or "code-switching".
- *Translation*: If applicable, provide the English translation.

# Examples
*Example 1:*

Input: "Tôi thích đi shopping vào cuối tuần."
- Detection Result: True
- Classification:
- "shopping": loanword
- Translation: "I like to go shopping on weekends."

*Example 2:*

Input: "Tôi thích đi mua sắm vào cuối tuần."
- Detection Result: False

# Notes

- Only proceed to classification and translation if English words are detected.
- Ensure accurate classification of English words to distinguish between loanwords and code-mixed-indexicality and code-switching.

---

Input: {input}"""

In [ ]:
examples = [
    "1000 followers Moon sẽ mở giveaway bộ 3000 từ vựng tiếng Anh thông dụng nhất cho cả nhà nha 🤩",
    "MÌNH CÓ THẮC MẮC ? LƯƠNG TRỢ LÝ Mọi người sẽ thuê trợ lý part time và full time. Lương mn trả cho trợ lý là bao nhiêu? Nếu công việc là: - Chụp ảnh, quản lý time nhắc hạn job - Phụ trách up bài, quản lý lịch job - Edit nếu cần (ko qá cần thiết cái này) - Đi theo event nếu cần thiết Mọi người share ít salary mn trả cho trợ lý vs đc khum. Mình tham khảo cái. Mình trả 150k/h với part time là cao hay thấp v mn ?",
    "First thread của em là lời kêu cứu nhờ giảng bài 😭 Em học accounting and finance năm nhất ấy ạ, mà chưa gì em đã không hiểu debit and credit rồi ;-; kiểu em không hiểu cái bản chất của nó ý (tại sao revenue lại là credit vậy) mọi người ạ ai đấy giúp em với huhu công đức vô lượng 😭🙏🏻",
    "Tại case của anh cũng hơi khác do anh mất thời gian khá dài nhưng anh nghĩ là xin intern rồi chờ cơ hội. Với lại này không phải là thành tựu em ơi vì cái này là kiểu partnership của bên công ty với đối tác để hiểu thêm về cách vận hành của TV bên Channel 7 thôi. Mấy cái này thì ai trong công ty làm mảng media cũng sẽ tổ chức thui á.",
    "ohh z chúc bà may mắn nhenn. Với lại tui hay xài mấy cái generic kiểu “i know your company stands for DEI and it deeply resonates w me kiểu z :))) nghe generic nhma tui nói z vẫn dc move forward vòng tiếp theo lun á",
    "Có ai cảm thấy đi học, học bao nhiêu cái framework đến lúc đi làm vớ phải start up nó lộn tùng phèo không? Đến lúc muốn purpose framework cũng khó nữa chưa kể là trong môi trường đó mình cũng dễ bị fomo nói là học nhanh nhưng mà chưa chắc đã sâu.",
    "Ở bài sửa này thì tôi đã thêm vào một số phương án thay thế cho việc \"strict punishments\" effective hơn và từ đó suy ra là tôi disagree với việc \"strict punishments\" là the only way.",
    "Stb số 1, bread factory là hàng bánh nhưng nước cũng ngon top 2",
    "Anh nhận định rằng có thể Examiner đã đánh giá cao mình ở 2 tiêu chí Lexical Resource và GR &Accuracy. Cụ thể khi được hỏi là Chính phủ nên trừng phạt những DN gây ô nhiễm hay khuyến khích khen thưởng họ nhìu hơn cho những hành động vì môi trường thì mình có trả lời “Punitive measures just act as a deterrent in the short term, not a feasible solution in the long run”.",
    "Book lịch 3 tháng, tới ngày hẹn bác sĩ gọi trước nửa tiếng cancel hẹn . Nge xong hết bệnh 🤣",
    "Congrats. Btw tớ có thể connect để có thể tìm job IT remote dễ hơn được không?",
    "không biết Việt Nam có cty nào in ship nội địa không chứ cái cờ đeo cổ hôm nay là mã customized graduation stole mấy công ty POD Việt Nam chạy mã cho nước ngoài nhiều lắm, màu vải này xưởng TQ có sẵn cả.",
    "Tham gia nhóm hóng hint free, tài liệu free và các gói ôn tủ các tháng 10-11-12 tại đây nhé",
    "Trời ơi dần dần Quang Hùng chiếm luôn cái tik tok của t luôn á. Xong cũng vì thế mà t biết nhiều cái hơn về Hùng. Có nhiều cái t thương dữ lắm. Rồi cả đến lúc được debut trong Best Five của ATSH nữa là t khóc luôn áaaaa😭😭. Lần đầu tiên stan idol mà cũng là idol Việt Nam nữa nên t cảm thấy tuyệt vời lắm",
    "Chatgpt should be your best friend. Xưa mình tự học code R với chatgpt để cook project trong 1 tuần.",
]

model_name = 'gpt-3.5-turbo'
model_name = 'meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo'
model_name = 'claude-3-5-sonnet-20240620'



with open('gpt4o-300.jsonl', 'w') as out_f:
    for example in examples:
        prompt = DETECTION_PROMPT.format(input=example)
        output = engine.generate([{
            'role': 'user', 'content': prompt
        }], model=model_name)
        json.dump({'output': output, 'input': example, 'model': model_name}, out_f, ensure_ascii=False)
        out_f.write('\n')
        print('======================')
        print('Input:', example)
        print('Output:', output)
        time.sleep(30)